#**Library**

In [ ]:
from PIL import Image
from PIL import JpegImagePlugin
from torch.nn import TransformerEncoder, TransformerEncoderLayer

import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd
from sklearn import preprocessing

import easydict
import math
import torch
import os
import glob
import time
import sys

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xvf '/content/drive/Shareddrives/예림 하이/label_yerim.tar'

# getAngle


In [ ]:
def getAngle(start, mid, end):
        """
        Calculate the angle between start and end
        :param start: start point [x, y]
        :param end: mid point [x, y]
        :param end: end point [x, y]
        :return: the clockwise angle from start to end
        """
        v1 = np.array([0,0])
        v2 = np.array([0,0])

        v1[0] = (start[0]-mid[0])
        v1[1] = (start[1]-mid[1])

        v2[0] = (end[0]-mid[0])
        v2[1] = (end[1]-mid[1])
       #  angle = np.arccos((v1[0]*v2[0]+v1[1]*v2[1])/(math.sqrt(v1[0]*v1[0] + v1[1]*v1[1])*math.sqrt(v2[0]*v2[0] + v2[1]*v2[1]))*math.pi/180)

        #a = (v1[0]*v2[0]+v1[1]*v2[1])
        k = v1[0]*v1[0] + v1[1]*v1[1]
        j = v2[0]*v2[0] + v2[1]*v2[1]
        if k==0 or j==0 :
          angle = 0
        else:
          b = math.sqrt(k)*math.sqrt(j)
          c = np.inner(v1,v2)/b
          angle = 0
          if c <= 1 and c >=-1 :
            angle = np.arccos(c)
        #  print(c)
        #  print('각도:')
       #   print(angle*180/math.pi)
        
        return angle*180/math.pi



In [ ]:
getAngle([1,0],[0,0],[1,1])

1
2


45.00000000000001

In [ ]:
getAngle([0,0], [0,0],[0,0])

0.0

In [ ]:
getAngle([1,1.717], [0,0],[1,0])

45.00000000000001

In [ ]:
# pose= np.load('label_yerim/label_9/cat-sitdown-098036.npy')
# #print(type(data))
# #print(data.shape)
# #print(data)

In [ ]:
# df = pd.read_csv('/content/label_yerim/label_8.csv')

# lab = df['action']
# filename = df['filename']

# label = lab.values.tolist()
# file_list = filename.values.tolist()

# print(df)

In [ ]:
act_dict = {
    '배를 보여주는 동작':6,
    '좌우로 뒹구는 동작':4,
    '좌우로 뒹굴음':4, 
    '허리를 아치로 세우는 동작':9,
    '그루밍하는 동작':3,
    '꼬리를 흔드는 동작':7,
    '그루밍함':3,
    '배를 보임':6,
    '걷거나 뜀 ':12,
    '옆으로 누워 있음':10,
    '꼬리를 흔든다':7, 
    '걷거나 달리는 동작':12,
    '납작 엎드림':1,
    '걷거나 뛰는 동작':12,
    '앞발로 꾹꾹 누름':8,
    '발을 숨기고 웅크리고 앉는 동작':2,
    '납작 엎드리는 동작':1,
    '허리를 아치로 세움':9, 
    '배를 보이는 동작':6,
    '머리를 들이대는 동작':5, 
    '팔을 뻗어 휘적거림':11,
    '앞발로 꾹꾹 누르는 동작':8,
    '옆으로 눕는 동작':10, 
    '앞발을 뻗어 휘적거리는 동작':11,
    '머리를 들이댐':5, 
    '발을 숨기고 웅크리고 앉음':2
}
def createLabel(label):
    label = int(act_dict[label]) -1
    return label

In [ ]:
print(createLabel('발을 숨기고 웅크리고 앉음'))


1


#**dataset.py**

In [ ]:
class trainDataset(torch.utils.data.Dataset):
    def __init__(self, data_path):
        train_dir_list = ['label_7', 'label_8', 'label_9']
        label_dir_list = ['label_7.csv', 'label_8.csv', 'label_9.csv']
        data_path = './label_yerim'

        video_list = []
        label = []

        for dir_idx in label_dir_list:
            df = pd.read_csv(os.path.join(data_path, dir_idx))
            lab = df['action']
            filename = df['filename']

            label_ = lab.values.tolist()
            label_ = list(map(lambda x: createLabel(x), label_))
            
            train_videos_ = filename.values.tolist()
            video_list.extend(train_videos_)
            label.extend(label_)
        # 여기까지가 csv파일 읽어오고, label이랑 filename받아오기(사실 한번에 해도 될거같은데..ㅜㅜㅅ ..편하게 함수쓰고 하드코딩 할래요오)

        train_videos = []
        for dir_idx in train_dir_list:
            path = os.path.join(data_path, dir_idx)
            for root, _, fnames in sorted(os.walk(path)):
                for fname in fnames:
                    file_name = fname.split('.npy')
                    if file_name[0] in video_list:
                        train_videos.append(os.path.join(path, fname))

        self.file_list = train_videos
        self.label_list = label

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
      video_data = np.load(self.file_list[idx])
      angle_list = []
      angle_set = [2, 26, 28]
      s, m ,f = angle_set[0], angle_set[1] , angle_set[2]

      '''
      코 머리 입 입끝 목 앞우관절 앞좌관절 앞우발끝 앞좌발끝 뒤우관절 뒤좌관절 뒤우발끝 뒤좌발끝 꼬리시작 꼬리끝
      0   2   4   6   8   10      12        14          16      18      20      22      24    26          28
      다리각도 정도랑/ 꼬리/ 머리 목 다리/
      '''

      for i in range(0, 30):
          start_point = list()
          mid_point = list()
          end_point = list()
          if video_data[i][s] != 0:
              start_point.append(video_data[i][s])
              start_point.append(video_data[i][s+1])
          else:
              if len(angle_list) == 0:
                  angle_list.append(0)
              else :
                angle_list.append(angle_list[-1])

              continue;
           ####   
          if video_data[i][m] != 0:
              mid_point.append(video_data[i][m])
              mid_point.append(video_data[i][m+1])
          else:
              if len(angle_list) == 0:
                  angle_list.append(0)
              else :
                angle_list.append(angle_list[-1])
                      
              continue;
          #####    
          if video_data[i][f] != 0:
              end_point.append(video_data[i][f])
              end_point.append(video_data[i][f+1])
          else:
              if len(angle_list) == 0:
                  angle_list.append(0)
              else :
                angle_list.append(angle_list[-1])                       
              continue;
          ######    
          angle = getAngle(start_point, mid_point, end_point)
          angle_list.append(angle)
        
      label = self.label_list[idx]
      item = (angle_list, label)
      print(item)
      return item

#**model.py**

In [ ]:
class TransformerModel(nn.Module):
  def __init__(self, input_size, nhead, hidden_size, n_layers, dropout=0.1):
    super(TransformerModel, self).__init__()


    self.model_type = 'Transformer'

    self.sequence_length = 30
    self.input_size = input_size

  ### 인코더
    encoder_layers = nn.TransformerEncoderLayer(
        d_model=input_size, nhead=nhead, dim_feedforward=hidden_size, dropout=dropout, activation='relu', batch_first=True
    )


    self.transformer_encoder = nn.TransformerEncoder(encoder_layer=encoder_layers, num_layers=n_layers)

##디코더
    self.customed_decoder = nn.Sequential(
        nn.Linear(30, 128),
        nn.ReLU(),
        nn.Linear(128, 128),
        nn.ReLU(),
        nn.Linear(128, 12)
    )

  def forward(self, angle_seq):
  #  print(angle_seq)
    features= angle_seq.float()
    features = torch.unsqueeze(angle_seq, axis=1)

    output = self.transformer_encoder(features)

    output = torch.squeeze(output, axis=1)
 #   print(output.shape)
    output = self.customed_decoder(output)
    
#    print(output)

    return output

#**train valid test**

In [ ]:
def train(dataloader, epoch):
    print('[Epoch %d]' % (epoch+1))

    criterion = nn.CrossEntropyLoss() 
    running_loss = 0.0

    start_time = time.time()
    for batch_idx, samples in enumerate(dataloader):
        # le = preprocessing.LabelEncoder()
        
        # labels = le.fit_transform(samples[1])
        labels = samples[1]
        labels = torch.tensor(labels)

        angle_list = torch.stack(samples[0], dim=1)


        angle_list, labels = angle_list.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad() 
        # forward + backward + optimize
        outputs = net(angle_list.float())
        loss = criterion(outputs, labels) 
      #  print(outputs, labels)
      #  print(loss)
        if not torch.isfinite(loss):
          print('WARNING: non-finite loss, ending training!')
          exit(1)

        loss.backward() 
        optimizer.step() 

        # print statistics
        running_loss += loss.item()
        if batch_idx % args.loss_interval == (args.loss_interval-1):
            elapsed = time.time() - start_time
            print('[%d, %5d] loss: %.6f in %.3f seconds' %
                  (epoch + 1, batch_idx + 1, running_loss / args.loss_interval, elapsed))
            
            running_loss = 0.0
            start_time = time.time()

        if args.split_training:
            if batch_idx > 10*args.loss_interval:
                break
    return running_loss / args.loss_interval

In [ ]:
def valid(dataloader, epoch):
    print('**Validation**')
    criterion = nn.CrossEntropyLoss()
    valid_loss = 0.0

    classes = (1,2,3,4,5,6,7,8,9,10,11,12)
    #,13,14,15,16,17,18,19,20,21,22,23,24,25, 26)
    class_correct = list(0. for i in range(12))
    class_total = list(0. for i in range(12))
    class_acc = list(0. for i in range(12))

    with torch.no_grad():
        for samples in dataloader:
            # le = preprocessing.LabelEncoder()

            # labels = le.fit_transform(samples[1])
            labels = samples[1]
            labels = torch.tensor(labels)

            angle_list = torch.stack(samples[0], dim=1)

            angle_list, labels = angle_list.to(device), labels.to(device)

            # forward + backward + optimize
            outputs = net(angle_list.float())
            loss = criterion(outputs, labels)

            # print statistics
            valid_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
     #       print("pred")
     #       print(predicted)
    #        raise ValueError
            c = (predicted == labels).squeeze()
   #         print(c)
   #         print(c.shape)
            for i in range(min(len(labels), args.batch_size)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

        hap = 0
        for i in range(12):
            if class_total[i] == 0:
                continue;
            class_acc[i] = 100 * class_correct[i] / class_total[i]
            print('Accuracy of %5s : %.2f %%' % (classes[i], class_acc[i]))
            hap += class_acc[i]

        valid_loss = valid_loss / len(dataloader)
        total_acc = hap / 26
        print('Accuracy of %5s : %.2f %%' % ('total', total_acc))
        print('Validation Loss: %.6f' % valid_loss)

      #  calculate valid best
        if total_acc > valid_best['total_acc']:
            valid_best['total_acc'] = total_acc
            valid_best['epoch'] = epoch + 1

            # save model
         #   os.makedirs('/content/drive/Shareddrives/예림하이/model', exist_ok=True)
            torch.save(net.state_dict(), '/content/drive/Shareddrives/예림 하이/model/transformer-action-best-detect.pth')

    return valid_loss, total_acc

#**<main**>

In [ ]:
import easydict
import pandas as pd
from sklearn.model_selection import train_test_split

args = easydict.EasyDict({
        "batch_size": 8,
        "epoch": 50,
        "loss_interval": 500,
        "split_training": False,
        "data_path": '/content/label_yerim',
        "model_name": 'transformer-action-detect.pth',
        "model_path": '/content/drive/Shareddrives/예림하이/model'})

dataset = trainDataset(args.data_path)
train_dataset, valid_dataset = train_test_split(dataset,  test_size=0.2, shuffle=True, random_state=34)



In [ ]:
if __name__ == "__main__":

    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model_save_path = os.path.join(args.model_path, args.model_name)


    input_size = 30
    nhead = 2
    hidden_size = 128
    n_layers = 4
    dropout = 0.1

    net = TransformerModel(input_size, nhead, hidden_size, n_layers, dropout)
    # net.load_state_dict(torch.load(model_save_path)) 
    net.to(device)
    net = net.float()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.00001, momentum=0.9, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)


    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=args.batch_size)

    valid_best = dict(epoch=0, total_acc=0)

    train_losses = []
    valid_losses = []
    valid_accuracy = []

    for epoch in range(0, args.epoch):
        epoch_start_time = time.time()
        net.train()
        loss = train(train_dataloader, epoch)

        net.eval()
        valid_loss, acc = valid(valid_dataloader, epoch)
        print("end of the epoch in %f seconds" % (time.time() - epoch_start_time))
        scheduler.step()
        
        train_losses.append(loss)
        valid_losses.append(valid_loss)
        valid_accuracy.append(acc)

        torch.save(net.state_dict(), '/content/drive/Shareddrives/예림 하이/transformer-action-detect.pth')

    print('[Best epoch: %d]' % valid_best['epoch'])
    print('Accuracy of %5s: %.2f %%' % ('total', valid_best['total_acc']))
    print('Done')

    net.eval()


In [ ]:
# import os
# import json
# import numpy as np

# def getJsonData(json_file):
#     with open(json_file, "r") as f:
#         data = json.load(f)
#     return data

# def getVideoData(data,video_name):
  
#     """
#     data format -> { "data": [ { "frame_index": int,
#                                  "skeleton": [{
#                                                 "pose": [x1, y1, x2, y2, ... ],
#                                                 "score": [1,1,1,...]
#                                              }]
#                                 },
#                                 { "frame_index": int,
#                                  "skeleton": [{
#                                                 "pose": [x1, y1, x2, y2, ... ],
#                                                 "score": [1,1,1,...]
#                                              }]
#                                 }, ...
#                             ]
#                      "label": str               # action
#                      "label_index":  int        # unique data index (not sure)
#                      }
#     """
  
#     video = data.get(video_name, None)
#     if video is None:
#         raise FileNotFoundError

#     # first category
#     vid_data = video['data']                # data contains frame_idx, skeleton(pose, score)
#     vid_label = video['label']              # action
#     vid_label_index = video['label_index']  # unique index

#     # init numpy array
#     pose_np = []
#     angle_list = []

#     # data category #frame 어떻게 자를지 결정하기
#     for frame in vid_data:
#         frame_idx = frame['frame_index']
#         skeleton = frame['skeleton']
#         pose = skeleton[0]['pose']


#         start_point = list()
#         mid_point = list()
#         end_point = list()
        

#         #6, 8, 10
#         if pose[11] != 0 :        
#           start_point.append(pose[11])
#           start_point.append(pose[12])
#         if pose[15] != 0 :
#           mid_point.append(pose[15])
#           mid_point.append(pose[16])
#         if pose[19] != 0 :
#           end_point.append(pose[19])
#           end_point.append(pose[20])

        
#         angle = getAngle(start_point, mid_point, end_point)
#         angle_list.append(angle)

#         score = skeleton[0]['score']

#         pose_np.append(_getPose(pose))

#     pose_np = np.array(pose_np)
#     print("angle : " + angle + "label : " + vid_label)
#     print(size(pose_np))
#     return angle_list, vid_label

# def _getPose(pose):
#     pose_reshape = []
#     for i, p in enumerate(pose):
#         if i % 2 == 1:
#             pos = [pose[i-1], pose[i]]
#             pose_reshape.append(pos)
#     return pose_reshape

# if __name__ == "__main__":
#     data = getJsonData("/content/label_1.json")
    
#     graph, label = getVideoData(data, "20201117_dog-walkrun-002803.mp4") #data 형식을 바꾸던지, data name별로 text에 저장하던디 해야할듯
    

